Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the dataset

In [2]:
dataset = pd.read_csv('US_Accidents_Dec20.csv')

In [9]:
dataset.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


temperature, windchill, hum, press, vis, winspeed, wea_cond,
y = severity

In [10]:
data_set = dataset.iloc[:500, [3,23,24,25,26,27,29]]
data_set.head()

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph)
0,3,36.9,NaN,91.0,29.68,10.0,NaN
1,2,37.9,NaN,100.0,29.65,10.0,NaN
2,2,36.0,33.3,100.0,29.67,10.0,3.5
3,3,35.1,31.0,96.0,29.64,9.0,4.6
4,2,36.0,33.3,89.0,29.65,6.0,3.5


In [11]:
data_set.head()

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph)
0,3,36.9,NaN,91.0,29.68,10.0,NaN
1,2,37.9,NaN,100.0,29.65,10.0,NaN
2,2,36.0,33.3,100.0,29.67,10.0,3.5
3,3,35.1,31.0,96.0,29.64,9.0,4.6
4,2,36.0,33.3,89.0,29.65,6.0,3.5


In [12]:
X = data_set.iloc[:,1:].values
Y = data_set.iloc[:,0].values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = "mean")
imputer.fit(X[:,:])
X[:,:] = imputer.transform(X[:,:])
print(X)

[[ 36.9         22.10545024  91.          29.68        10.
    9.6556962 ]
 [ 37.9         22.10545024 100.          29.65        10.
    9.6556962 ]
 [ 36.          33.3        100.          29.67        10.
    3.5       ]
 ...
 [ 32.          28.7         93.          30.2         10.
    3.5       ]
 [ 32.          25.6         88.          30.2          4.
    6.9       ]
 [ 32.          24.8         88.          30.2          5.
    8.1       ]]


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [14]:
#Random Forest
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 12, criterion = 'entropy', random_state=0)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)


0.68

In [20]:
def current_location():
    import requests
    r = requests.get('https://api.ipdata.co?api-key=31f7a5f46401e33ac72fca8d9242a023eea1ca8f4ff562f60b315a29').json()   
    location = [r['city'], r['country_name']]

    return location

location = current_location()
#location = ['New York', 'US']

def weather_conditions(location):    
    from pyowm.owm import OWM
    owm = OWM('444fb686d7df51b68b9504b0eb2778f7')
    mgr = owm.weather_manager()
    place = str(location[0]+','+location[1])    
    observation = mgr.weather_at_place(place)
    weather = observation.weather
    #observation = mgr.weather_at_place('New York','US')
    weather.status           
    weather.detailed_status

    #temp
    temp_k = weather.temperature()
    Celsius = temp_k['temp'] - 273.15
    Fahrenheit_t = Celsius*(9/5) + 32

    #wind chill
    temp_windchill = temp_k['feels_like'] - 273.15
    Fahrenheit_wc = Celsius*(9/5) + 32

    #humidity
    humidity = weather.humidity

    #pressure
    press = weather.pressure
    pressure = press['press']/51.715

    #visibility
    vis = weather.visibility_distance
    visibility = vis/1609

    #wind speed
    wind_speed = weather.wind()['speed']

    #parameters
    parameters = [Fahrenheit_t,Fahrenheit_wc,humidity,pressure,visibility,wind_speed]
    
    return parameters


In [22]:
parameters = weather_conditions(location)
severity = classifier.predict([parameters])

In [25]:
if(severity==1):
    print("Probability of accident is too low")
    print()
    print("Speed Limit must be below 80kmph")
elif(severity==2):
    print("Probability of accident is moderate")
    print()
    print("Speed Limit must be below 60kmph")

else:
    print("This location is prone to accidents⚠")
    print()
    print("Drive Safely!")
    print()
    print("Speed Limit must be below 45kmph")

Probability of accident is moderate

Speed Limit must be below 60kmph
